In [0]:
# Load the Silver data we created earlier so the test can see it
silver_df = spark.read.parquet("/Volumes/workspace/default/uk_land_registry/silver_engineered_parquet")

# Now run the test
def test_data_quality(df):
    from pyspark.sql.functions import col
    null_count = df.filter(col("Price").isNull()).count()
    col_count = len(df.columns)
    
    if null_count == 0 and col_count >= 5:
        print("Unit Test Passed: Data is clean and schema is correct.")
    else:
        print(f"Unit Test Failed: Found {null_count} nulls.")

test_data_quality(silver_df)

Unit Test Passed: Data is clean and schema is correct.
